In [2]:
%cd /data/bruingjde/on-going/SNAM2021-code/

from constants import *

/data/bruingjde/on-going/SNAM2021-code


ModuleNotFoundError: No module named 'tlp'

In [2]:
df = networks.copy()
df['index'] = df.index
df['edge type'] = df['index'].isin(hypergraph_indices).replace(
  {True: 'persistent', False: 'event'}
)
df['source'] = df['source'].apply(lambda x: f'\cite{{{x}}}')
df['domain'] = df['category'].replace(
  {'social': 'Social', 'information': 'Inf.', 'technological': 'Tech.'})
df.drop(columns=['old_category', 'category'], inplace=True)

In [3]:
# def get_size(network: int):
#   edgelist = pd.read_pickle(f'data/{network:02}/edgelist.pkl')
#   graph = nx.from_pandas_edgelist(edgelist, create_using=nx.MultiGraph)
    
#   return pd.Series(
#     dict(nodes=graph.number_of_nodes(), edges=graph.number_of_edges()),
#     name=network
#   )

# size = tlp.ProgressParallel(n_jobs=network_count, total=network_count)(
#   joblib.delayed(get_size)(network) for network in network_indices
# )
# size = pd.DataFrame(size) 
# size.rename(dict(nodes='n', edges='m'), inplace=True)
size = pd.read_pickle('code/figures/size.pkl')

In [4]:
stats = dict()
for network in tqdm(network_indices):
  with open(f'data/{network:02}/stats.json') as file:
    stats[network] = json.load(file)
stats = pd.DataFrame.from_dict(stats, orient='index')
stats.rename(columns={'density (nx.Graph)': 'density', 'degree assortativity (nx.Graph)': 'degree assortativity'}, inplace=True)

columns = ['density', 'degree assortativity', 
           'average clustering coefficient']

stats = stats[columns].copy()

# stats.drop(columns=['nodes', 'edges', 'density (nx.MultiGraph)', 'fraction edges in GC'], inplace=True)

  0%|          | 0/26 [00:00<?, ?it/s]

In [5]:
diameter = pd.Series(get_diameter(), name='diameter')

  0%|          | 0/26 [00:00<?, ?it/s]

In [6]:
def convert_int_to_short(x: int):
  if x > 2000000: return f'{x/1000000:.0f}M'
  elif x > 1000000: return f'{x/1000000:.1f}M'
  elif x > 2000: return f'{x/1000:.0f}K'
  elif x > 1000: return f'{x/1000:.1f}K'
  else: return str(x)
  
def scientific_notation(x): 
  x = f'{x:.0e}'
  coefficient = x[0]
  if '-' in x:
    exponent = '-' + x[-1]
  else:
    exponent = x[1]
  return f'${coefficient}\!\cdot\!10^{{{exponent}}}$'

def thousand_sep(x):
  return f'{x:,}'

In [7]:
result = pd.concat([df, stats, size, diameter], axis=1)
result.dropna(inplace=True)
result = result.astype(dict(diameter=int))
result.reset_index(inplace=True, drop=True)
result.index = result.index + 1
result.sort_values('nodes', inplace=True)
result['density'] = result['density'].apply(scientific_notation)
result['nodes'] = result['nodes'].apply(thousand_sep)
result['edges'] = result['edges'].apply(thousand_sep)
result['degree assortativity'] = result['degree assortativity'].round(2)
result['average clustering coefficient'] = (
  result['average clustering coefficient'].round(2))

In [8]:
columns = [
  'label', 'domain', 'edge type', 'nodes', 'edges', 'density', 
  'degree assortativity', 'average clustering coefficient', 'diameter', 
  'source']

In [9]:
table = result[columns].copy()
table.rename(
  {
    'source': ' ',
    'degree assortativity': 'd.a.', 
    'average clustering coefficient': 'a.c.c.',
    'diameter': 'diam.',
    'nodes': 'nodes ($n$)',
    'edges': 'edges ($m$)'
  },
  axis='columns',
  inplace=True)

In [10]:
latex_table = table.to_latex(
  column_format='l@{\hspace{1em}}c@{\hspace{1em}}c@{\hspace{0.5em}}r@{\hspace{1em}}r@{\hspace{1em}}c@{\hspace{1em}}c@{\hspace{1em}}c@{\hspace{1em}}c@{\hspace{0.5em}}c',
  index=False,
  caption=(
    "Networks used in this work. "
    "The following abbreviations are used in the columns; "
    "d.a.: degree assortativity, acc: average clustering coefficient, diam.: diameter. "
    "In the column `domain', Technological is abbreviated to Tech. and Information to Inf."
  ),
  label='table:datasets',
  escape=False,
  multicolumn=False,
)
print(latex_table)

\begin{table}
\centering
\caption{Networks used in this work. The following abbreviations are used in the columns; d.a.: degree assortativity, acc: average clustering coefficient, diam.: diameter. In the column `domain', Technological is abbreviated to Tech. and Information to Inf.}
\label{table:datasets}
\begin{tabular}{l@{\hspace{1em}}c@{\hspace{1em}}c@{\hspace{0.5em}}r@{\hspace{1em}}r@{\hspace{1em}}c@{\hspace{1em}}c@{\hspace{1em}}c@{\hspace{1em}}c@{\hspace{0.5em}}c}
\toprule
label & domain &  edge type & nodes ($n$) & edges ($m$) &             density &  d.a. &  a.c.c. &  diam. &                          \\
\midrule
 Rado & Social & persistent &         167 &      82,927 & $2\!\cdot\!10^{-1}$ & -0.29 &    0.59 &      5 &     \cite{Michalski2011} \\
   UC &   Inf. & persistent &         899 &      33,720 & $2\!\cdot\!10^{-2}$ & -0.09 &    0.07 &      6 &        \cite{Opsahl2013} \\
   EU & Social & persistent &         986 &     332,334 & $3\!\cdot\!10^{-2}$ & -0.03 &    0.41 &      

In [13]:
result.groupby(['domain'])[['density', 'degree assortativity', 'average clustering coefficient', 'diameter']].agg(['mean', 'std'])

degree assortativity           average clustering coefficient  \
                       mean       std                           mean   
domain                                                                 
Inf.               0.001667  0.193227                       0.273333   
Social            -0.043333  0.163887                       0.272778   
Tech.             -0.115000  0.035355                       0.180000   

                   diameter            
             std       mean       std  
domain                                 
Inf.    0.329768   8.333333  3.265986  
Social  0.213454  13.444444  4.973130  
Tech.   0.000000  14.000000  4.242641